In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPoint,Point
from shapely.ops import nearest_points

fires_df=pd.read_csv('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/Datasets/telangana_fires.csv')

import warnings
warnings.filterwarnings("ignore")

Upsampling is done to meet the MODIS resolution requiremnt and comare with telangana fires dataset, otherwise dataset is very large. And to solve that purpose, I assume some generalization related to pixel value when downsampling, so it's give descent results.

In [2]:
path='D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/AvinashMahech/esri_lulc_telangana.tif'
import rasterio
from rasterio.enums import Resampling

downsacle_factor = 1/25    # Downsampling by 25 to meet the MODIS resolution requirements 

with rasterio.open(path,'r') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * downsacle_factor),
            int(dataset.width * downsacle_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [3]:
data.shape

(1, 1809, 1945)

In [6]:
geo_transform=transform
geo_transform

Affine(250.0051413881748, 0.0, 97906.8554,
       0.0, -250.1160862354891, 2204414.5023)

In [7]:
data=data.reshape((1809,1945))

In [8]:
data.shape

(1809, 1945)

In [9]:
import numpy as np
np.unique(data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=uint8)

In [10]:
## Find out the class 5
results=np.where(data==5)
results

(array([   0,    0,    0, ..., 1733, 1733, 1733], dtype=int64),
 array([481, 482, 483, ..., 418, 419, 420], dtype=int64))

In [11]:
# Convert Mulitple points to check nearest point to ESRI LULC value location.
points=MultiPoint([(x,y) for x,y in zip(fires_df.longitude,fires_df.latitude)])

In [12]:
points.wkt

'MULTIPOINT (80.5314 18.3777, 80.9893 17.5898, 81.16119999999999 17.6028, 80.3135 17.5241, 80.3404 17.7571, 80.34650000000001 17.7513, 80.3616 17.7421, 80.342 17.7385, 80.8069 17.9539, 78.38549999999999 19.0243, 77.97620000000001 18.6166, 77.99720000000001 18.6144, 80.9606 17.3005, 80.9481 17.3021, 80.7641 17.2146, 78.7864 16.0515, 78.6494 18.1696, 78.6588 18.171, 77.9877 16.6521, 80.1516 17.8572, 80.8198 17.1496, 80.3436 17.6176, 80.6688 18.02, 79.8627 16.8424, 80.0628 17.7721, 79.9015 18.1545, 80.03959999999999 18.146, 79.83499999999999 18.1452, 79.9451 18.2986, 79.93559999999999 18.2973, 79.87220000000001 18.5915, 77.8151 18.2036, 78.6435 18.9335, 79.3843 18.5702, 80.86190000000001 17.8345, 80.1955 17.7498, 80.4004 18.5093, 80.87820000000001 17.1789, 80.64239999999999 18.1029, 80.5292 18.3891, 80.3583 17.6913, 80.36839999999999 17.6928, 78.809 16.0638, 78.0753 16.8283, 77.8498 17.7252, 78.0818 17.8771, 78.8013 16.0664, 79.9819 17.9643, 80.0558 18.4912, 78.2869 17.4889, 79.0313 17.9,

In [30]:
tr=geo_transform
from pyproj import Proj, transform
inProj = Proj(init='epsg:32644')  #ESRI LULC Projection
outProj = Proj(init='epsg:4326')  #MODIS Projection
fires_df['class']=0

j=0
for i in range(len(results[0])):
    # Geo Transformation Matrix
    x=tr[1]*results[1][i]+tr[2]*results[0][i]+(tr[1]+tr[2])*0.5+tr[0] 
    y=tr[4]*results[1][i]+tr[5]*results[0][i]+(tr[4]+tr[5])*0.5+tr[3]
    x_,y_ = transform(inProj,outProj,x,y)
    p=nearest_points(points,Point(x_,y_))[0]
    confidence=fires_df.loc[(fires_df.latitude==p.y)&(fires_df.longitude==p.x)]['confidence']
    if confidence.values[0]>45:   # Threshold of 45 percent confidence
        fires_df.loc[j,'class']=1
    j+=1

In [31]:
df=fires_df.loc[:,['fireID','class']]
df.head()

,fireID,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [32]:
output2=df.to_json('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/AvinashMahech/output2.json',orient="index")
output2